In [9]:
import qiskit
import bqskit

# Get the version of Qiskit
qiskit_version = qiskit.__qiskit_version__

qiskit_version

print(bqskit.__version__, qiskit.__version__)

1.1.2 0.45.0


下面一个模块是创建coupling graph。原始的代码在route forcing文件夹里面。目前（24-09-25）是能跑的，有问题的话重新导入代码。

In [12]:
# Re-import necessary libraries after code reset
import networkx as nx
import pandas as pd
import numpy
from typing import Sequence

# 创建一个只包含单元内连接的程序
def create_coupling_graph_in_unit_only():
    G = nx.Graph()

    # 添加256个点
    for unit in range(4):
        for row in range(8):
            for col in range(8):
                node = f'{unit}{row}{col}'
                G.add_node(node)

    # 单元内连接 (in_unit)
    for unit in range(4):
        for row in range(8):
            for col in range(8):
                node = f'{unit}{row}{col}'

                # 邻近点连接 (in-unit)
                if col < 7:  # 向右连接
                    right_node = f'{unit}{row}{col + 1}'
                    G.add_edge(node, right_node, weight=1, connection_type='in_unit')

                if row < 7:  # 向下连接
                    down_node = f'{unit}{row + 1}{col}'
                    G.add_edge(node, down_node, weight=1, connection_type='in_unit')

    return G

# 使用 Floyd-Warshall 算法计算点到点的最短距离矩阵
def calculate_distance_matrix_floyd_warshall(graph):
    # 使用 networkx 的 floyd_warshall 函数
    length = dict(nx.floyd_warshall(graph, weight='weight'))
    
    # 转换为矩阵形式，便于输出或分析
    nodes = sorted(graph.nodes())
    num_nodes = len(nodes)
    distance_matrix = []

    for i in range(num_nodes):
        row = []
        for j in range(num_nodes):
            row.append(length[nodes[i]][nodes[j]])
        distance_matrix.append(row)
    
    return distance_matrix, nodes

# 创建仅包含单元内连接的 coupling graph
G_in_unit_only = create_coupling_graph_in_unit_only()

# 手动添加单元间连接
edges_to_add = [
    ('007', '100', 5),
    ('017', '110', 5),
    ('027', '120', 5),
    ('037', '130', 5),
    ('047', '140', 5),
    ('057', '150', 5),
    ('067', '160', 5),
    ('077', '170', 5),
    ('070', '200', 5),
    ('071', '201', 5),
    ('072', '202', 5),
    ('073', '203', 5),
    ('074', '204', 5),
    ('075', '205', 5),
    ('076', '206', 5),
    ('077', '207', 5),
    ('170', '300', 5),
    ('171', '301', 5),
    ('172', '302', 5),
    ('173', '303', 5),
    ('174', '304', 5),
    ('175', '305', 5),
    ('176', '306', 5),
    ('177', '307', 5),
    ('207', '300', 5),
    ('217', '310', 5),
    ('227', '320', 5),
    ('237', '330', 5),
    ('247', '340', 5),
    ('257', '350', 5),
    ('267', '360', 5),
    ('277', '370', 5),
]

# 添加单元间连接
for edge in edges_to_add:
    G_in_unit_only.add_edge(edge[0], edge[1], weight=edge[2], connection_type='inter_unit')

# 使用 Floyd-Warshall 算法计算点到点的最短距离矩阵（包含单元间连接）
distance_matrix_in_unit_only, nodes_in_unit_only = calculate_distance_matrix_floyd_warshall(G_in_unit_only)

# 输出结果
distance_matrix = pd.DataFrame(distance_matrix_in_unit_only, index=nodes_in_unit_only, columns=nodes_in_unit_only)



In [13]:
print(distance_matrix)
print(G_in_unit_only.edges)

     000  001  002  003  004  005  006  007  010  011  ...  366  367  370  \
000    0    1    2    3    4    5    6    7    1    2  ...   36   37   31   
001    1    0    1    2    3    4    5    6    2    1  ...   35   36   30   
002    2    1    0    1    2    3    4    5    3    2  ...   34   35   29   
003    3    2    1    0    1    2    3    4    4    3  ...   33   34   28   
004    4    3    2    1    0    1    2    3    5    4  ...   32   33   27   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
373   34   33   32   31   30   29   28   27   33   32  ...    4    5    3   
374   35   34   33   32   31   30   29   28   34   33  ...    3    4    4   
375   36   35   34   33   32   31   30   29   35   34  ...    2    3    5   
376   37   36   35   34   33   32   31   30   36   35  ...    1    2    6   
377   38   37   36   35   34   33   32   31   37   36  ...    2    1    7   

     371  372  373  374  375  376  377  
000   32   33   34   35   36   37 

ok,看起来没问题

后面的coupling graph要求的是`[turple[int,int]]`的形式。好吧那么我们不能把任务留到后面去

亲啊，你设置了256个index，不能冲到377啊

In [14]:
def octal_to_decimal(octal_str):
    # 通过公式 64*第一位 + 8*第二位 + 第三位 转换为十进制
    return 64 * int(octal_str[0]) + 8 * int(octal_str[1]) + int(octal_str[2])
    

In [15]:
test_cpg =nx.Graph()


for edge in G_in_unit_only.edges:
    # 将每个节点从八进制字符串转换为十进制
    node1 = octal_to_decimal(edge[0])
    node2 = octal_to_decimal(edge[1])
    test_cpg.add_edge(node1, node2)

print(list(test_cpg.edges))

[(0, 1), (0, 8), (1, 2), (1, 9), (8, 9), (8, 16), (2, 3), (2, 10), (9, 10), (9, 17), (3, 4), (3, 11), (10, 11), (10, 18), (4, 5), (4, 12), (11, 12), (11, 19), (5, 6), (5, 13), (12, 13), (12, 20), (6, 7), (6, 14), (13, 14), (13, 21), (7, 15), (7, 64), (14, 15), (14, 22), (15, 23), (15, 72), (64, 65), (64, 72), (16, 17), (16, 24), (17, 18), (17, 25), (18, 19), (18, 26), (19, 20), (19, 27), (20, 21), (20, 28), (21, 22), (21, 29), (22, 23), (22, 30), (23, 31), (23, 80), (72, 73), (72, 80), (24, 25), (24, 32), (25, 26), (25, 33), (26, 27), (26, 34), (27, 28), (27, 35), (28, 29), (28, 36), (29, 30), (29, 37), (30, 31), (30, 38), (31, 39), (31, 88), (80, 81), (80, 88), (32, 33), (32, 40), (33, 34), (33, 41), (34, 35), (34, 42), (35, 36), (35, 43), (36, 37), (36, 44), (37, 38), (37, 45), (38, 39), (38, 46), (39, 47), (39, 96), (88, 89), (88, 96), (40, 41), (40, 48), (41, 42), (41, 49), (42, 43), (42, 50), (43, 44), (43, 51), (44, 45), (44, 52), (45, 46), (45, 53), (46, 47), (46, 54), (47, 55),

然后下面开始是正式地使用bqskit进行一些运行。首先我们测试各个模块能不能运行，代码从教程里面抄。

In [16]:
from bqskit import Circuit
circuit = Circuit.from_file('qasm/heisenberg16.qasm')

In [17]:
print("Circuit Statistics")
print("Gate Counts:", circuit.gate_counts)
print("Logical Connectivity:", circuit.coupling_graph)

Circuit Statistics
Gate Counts: {XGate: 8, RZGate: 180, HGate: 240, RXGate: 240, CNOTGate: 360}
Logical Connectivity: CouplingGraph({(0, 1), (9, 10), (13, 14), (1, 2), (10, 11), (3, 4), (12, 13), (2, 3), (6, 7), (4, 5), (8, 9), (11, 12), (5, 6), (14, 15), (7, 8)})


我们发现circuit是内嵌在.qasm文件里面的。经过进一步调查，我们首先要手搓machine model。

In [18]:
from bqskit.compiler import MachineModel

#定义qubits
qubits_256 = list(range(256))


In [19]:
connections = (list(test_cpg.edges))

print(connections)

[(0, 1), (0, 8), (1, 2), (1, 9), (8, 9), (8, 16), (2, 3), (2, 10), (9, 10), (9, 17), (3, 4), (3, 11), (10, 11), (10, 18), (4, 5), (4, 12), (11, 12), (11, 19), (5, 6), (5, 13), (12, 13), (12, 20), (6, 7), (6, 14), (13, 14), (13, 21), (7, 15), (7, 64), (14, 15), (14, 22), (15, 23), (15, 72), (64, 65), (64, 72), (16, 17), (16, 24), (17, 18), (17, 25), (18, 19), (18, 26), (19, 20), (19, 27), (20, 21), (20, 28), (21, 22), (21, 29), (22, 23), (22, 30), (23, 31), (23, 80), (72, 73), (72, 80), (24, 25), (24, 32), (25, 26), (25, 33), (26, 27), (26, 34), (27, 28), (27, 35), (28, 29), (28, 36), (29, 30), (29, 37), (30, 31), (30, 38), (31, 39), (31, 88), (80, 81), (80, 88), (32, 33), (32, 40), (33, 34), (33, 41), (34, 35), (34, 42), (35, 36), (35, 43), (36, 37), (36, 44), (37, 38), (37, 45), (38, 39), (38, 46), (39, 47), (39, 96), (88, 89), (88, 96), (40, 41), (40, 48), (41, 42), (41, 49), (42, 43), (42, 50), (43, 44), (43, 51), (44, 45), (44, 52), (45, 46), (45, 53), (46, 47), (46, 54), (47, 55),

In [20]:
# 定义支持的量子门
supported_gates = {
    "single_qubit": ["h", "x", "y", "z", "s", "t", "rx", "ry", "rz"],
    "two_qubit": ["cx", "cz"]
}

In [21]:
# 创建 MachineModel 实例
Test_Field_mcm = MachineModel(
    num_qudits=256,
    coupling_graph=list(test_cpg.edges),
    gate_set = None,
    radixes = [2] * 256
)


在经过了艰苦的奋斗之后，你终于成功地搓出了machine model。可喜可贺可喜可贺。就是时间实在拖得长了一些......

In [25]:
print("Logical Connectivity:", Test_Field_mcm.coupling_graph)

Logical Connectivity: CouplingGraph({(137, 145), (247, 255), (96, 104), (67, 68), (18, 26), (99, 100), (229, 237), (8, 9), (40, 41), (170, 178), (251, 252), (129, 137), (100, 101), (132, 133), (111, 119), (192, 193), (221, 229), (41, 42), (73, 74), (203, 211), (133, 134), (162, 170), (14, 15), (144, 152), (225, 226), (74, 75), (103, 111), (106, 107), (236, 244), (195, 203), (166, 167), (226, 227), (177, 185), (107, 108), (136, 144), (118, 126), (228, 236), (48, 49), (77, 85), (80, 81), (210, 218), (140, 141), (169, 177), (151, 159), (232, 233), (81, 82), (110, 118), (243, 251), (173, 174), (202, 210), (22, 23), (51, 59), (233, 234), (114, 115), (143, 151), (206, 207), (235, 243), (56, 128), (147, 148), (135, 192), (57, 129), (240, 241), (25, 33), (181, 182), (17, 25), (123, 195), (214, 215), (21, 22), (50, 58), (113, 114), (205, 206), (54, 55), (146, 147), (24, 32), (84, 92), (176, 184), (179, 180), (28, 29), (122, 194), (88, 89), (117, 125), (120, 121), (180, 181), (209, 217), (29, 30

然后我写一个新的`_get_distance()`方法：这里使用已经加载的`distance_matrix`而无需每次进行计算。

（原型在sabre.py里面。我还不知道应该怎么优雅地写自己的basepass，只好先罗列着）

In [ ]:
def _get_distance(
    self,
    logical_qudits: Sequence[int],
    pi: list[int],
    D: pd.DataFrame,  
    # 将 D 的类型更改为 pd.DataFrame。仅仅为了适配这里的代码，本质任然是一个二维矩阵。
) -> float:
    """计算连接逻辑量子位所需交换次数的期望值。"""
    min_term = np.inf
    for q in logical_qudits:
        term = 0.0
        for p in logical_qudits:
            if p == q:
                continue
            # 使用 DataFrame 的 .loc 方法获取距离
            term += D.loc[pi[q], pi[p]]
        min_term = min(term, min_term)
    return min_term

然后尝试重新写一下`_score_swap`方法。

In [3]:
import bqskit

In [10]:
from bqskit.ir import Circuit
from bqskit.compiler.basepass import BasePass

In [12]:
from bqskit.compiler import PassManager

ImportError: cannot import name 'PassManager' from 'bqskit.compiler' (/home/xuwentao/anaconda3/envs/bqskit/lib/python3.10/site-packages/bqskit/compiler/__init__.py)

p.s.下面那段代码是gpt给我写的。我真的不知道是什么意思.jpg。暂时留着反正会报错。

In [22]:
import asyncio
from bqskit.compiler import BasePass, PassManager
from bqskit.ir.circuit import Circuit
from bqskit.ir.gates import U3Gate, CXGate
from bqskit.passes.mapping import GeneralizedSabreLayoutPass, GeneralizedSabreRoutingPass
from bqskit.qis.graph import CouplingGraph

class MyCustomPass(BasePass):
    async def run(self, circuit: Circuit, data) -> None:
        # 这里可以添加自定义的处理逻辑，例如打印电路信息
        print(f"电路包含 {len(circuit.operations)} 个操作。")

async def main():
    # 定义量子比特数量
    num_qudits = 5

    # 创建一个包含num_qudits个量子比特的电路
    circuit = Circuit(num_qudits)

    # 添加大约100个随机的量子门来测试算法性能
    for _ in range(50):
        # 添加单量子比特门U3Gate到随机的量子比特上
        qudit = _ % num_qudits  # 简单地循环使用量子比特索引
        circuit.append_gate(U3Gate(), [qudit])

        # 添加双量子比特门CXGate到随机的量子比特对上
        control = _ % num_qudits
        target = (_ + 1) % num_qudits
        circuit.append_gate(CXGate(), [control, target])

    # 定义目标量子硬件的耦合图，这里使用全连接图作为示例
    coupling_graph = CouplingGraph.full(num_qudits)

    # 创建PassManager并添加Pass
    pass_manager = PassManager([
        MyCustomPass(),
        GeneralizedSabreLayoutPass(coupling_graph),
        GeneralizedSabreRoutingPass(coupling_graph),
    ])

    # 运行PassManager
    await pass_manager.run(circuit)

# 运行异步主函数
asyncio.run(main())


ImportError: cannot import name 'PassManager' from 'bqskit.compiler' (/home/xuwentao/anaconda3/envs/bqskit/lib/python3.10/site-packages/bqskit/compiler/__init__.py)